# 🌐 Agents using LangGraph

## Introduction

Welcome to this comprehensive tutorial on building **AI Agents** using **LangGraph**!

### What is LangGraph?

**LangGraph** is a Python framework developed by the LangChain team that allows you to build **stateful, multi-step workflows** involving language models, tools, and external APIs. It structures logic as a **directed graph** where:

- **Nodes** represent computational steps (e.g., calling an LLM or using a tool)
- **Edges** define how the workflow proceeds based on output or state

### Why Use LangGraph?

LangGraph is ideal for:

| Use Case | Description |
|----------|-------------|
| 🔄 **Multi-turn Chatbots** | Conversations with memory |
| 🌳 **Decision Trees** | Branching logic based on LLM outputs |
| 🛠️ **Complex Tool-Using Agents** | Agents that call multiple external services |
| 📊 **ETL Pipelines** | Data enrichment and transformation workflows |
| 💬 **Modular Conversational Flows** | Reusable conversation components |

### LangGraph vs LangChain

```
┌────────────────────────────────────────────────────────────────┐
│                        LangChain                               │
│   Linear chains, prompts, memory, tools, basic agents          │
│                           │                                    │
│                           ▼                                    │
│                      LangGraph                                 │
│   Graph-based workflows, stateful agents, branching logic,     │
│   dynamic flow control, persistent state across steps          │
└────────────────────────────────────────────────────────────────┘
```

### What We'll Cover

1. **Setting up Ollama** - Connect to locally running LLM
2. **LangGraph Agent Basics** - Create agents without tools
3. **External Tools** - Integrate DuckDuckGo search and Wikipedia
4. **Custom Tools** - Build your own tools with `@tool` decorator
5. **Memory** - Add conversational memory to your agents

---

## 📦 Installation

First, let's install the required packages:

- `langchain` - Core LangChain framework
- `langgraph` - Graph-based workflow framework
- `langchain-ollama` - Ollama integration for local LLMs
- `langchain-community` - Community tools and integrations
- `duckduckgo-search` - Free search API (no API key needed!)
- `wikipedia` - Wikipedia API wrapper

In [1]:
# !pip install langchain langgraph langchain-ollama langchain-community duckduckgo-search ddgs wikipedia -q

---

## 🦙 Setting Up Ollama

**Ollama** allows you to run LLMs locally on your machine. Before running this notebook:

1. Install Ollama from [ollama.ai](https://ollama.ai)
2. Pull a model: `ollama pull llama3.2` (or `mistral`, `qwen2.5`, etc.)
3. Ensure Ollama is running: `ollama serve`

### Why Local LLMs?

| Benefit | Description |
|---------|-------------|
| 🔒 **Privacy** | Your data never leaves your machine |
| 💰 **Cost** | No API fees - completely free |
| ⚡ **Speed** | No network latency for inference |
| 🔧 **Control** | Full control over model and parameters |

In [3]:
from langchain_ollama import ChatOllama

# Initialize the Ollama LLM
# You can change the model to any model you have pulled in Ollama
llm = ChatOllama(
    model="llama3.2",  # or "mistral", "qwen2.5", "phi3", etc.
    temperature=0,      # 0 for deterministic, higher for more creative
)

# Test the connection
response = llm.invoke("Say 'Hello, I am ready to be your LangGraph Agent!' in one line.")
print(f"✅ Ollama is working!\n\n🤖 LLM Response: {response.content}")

✅ Ollama is working!

🤖 LLM Response: "Hello, I am ready to be your LangGraph Agent!"


---

## 🧠 LangGraph Agent Basics

Let's start by creating a **basic agent** using LangGraph. For this initial example, we won't integrate any external tools, meaning the agent will rely solely on its internal reasoning capabilities.

### Understanding the ReAct Agent

LangGraph uses the **ReAct** (Reasoning + Acting) pattern:

```
Question ──▶ Thought ──▶ Action ──▶ Observation ──▶ ... ──▶ Final Answer
     │           │           │            │
     │     "I need to    "Search"    "Results..."     
     │      search..."                    │
     │                                    │
     └────────────────────────────────────┘
                    (Loop until solved)
```

### Basic Agent Graph Structure

Without tools, the graph is simple:

```
    __start__
        │
        ▼
      agent
        │
        ▼
     __end__
```

In [4]:
from langgraph.prebuilt import create_react_agent
from pprint import pprint

# Create a basic ReAct agent without tools
tools = []  # No tools for now
agent_executor = create_react_agent(llm, tools)

print("✅ Basic LangGraph Agent created!")

✅ Basic LangGraph Agent created!


/tmp/ipykernel_1201/2861338313.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)


In [5]:
def run_agent(query: str):
    """Function to run the agent with a user query and display traces."""
    state = agent_executor.invoke({"messages": [("user", query)]})
    
    print("\n" + "="*60)
    print("📋 AGENT TRACE:")
    print("="*60)
    
    for i, msg in enumerate(state["messages"]):
        msg_type = msg.type.upper()
        content = msg.content.strip() if hasattr(msg, 'content') and msg.content else "[Tool Call]"
        print(f"{i+1}. [{msg_type}] {content[:200]}{'...' if len(content) > 200 else ''}")
        print("-"*40)
    
    return state["messages"][-1].content

### Testing the Basic Agent

Let's ask some questions that the agent can answer from its training data:

In [6]:
# Question about technology - should work from training data
query = "Explain what a neural network is in simple terms."
answer = run_agent(query)

print("\n" + "="*60)
print("🎯 FINAL ANSWER:")
print("="*60)
print(answer)


📋 AGENT TRACE:
1. [HUMAN] Explain what a neural network is in simple terms.
----------------------------------------
2. [AI] A neural network is a computer system that's inspired by the way our brains work.

Imagine you have a bunch of interconnected nodes (like little boxes) that can learn and improve over time, just like ...
----------------------------------------

🎯 FINAL ANSWER:
A neural network is a computer system that's inspired by the way our brains work.

Imagine you have a bunch of interconnected nodes (like little boxes) that can learn and improve over time, just like how our brains do. Each node receives input from other nodes, processes it, and then sends the output to other nodes.

In a neural network, these nodes are called "neurons" (yes, named after the human brain cells!). They're connected by lines called "connections," which allow them to communicate with each other.

When you give a neural network some data, like pictures or words, it tries to figure out patterns

In [7]:
# Simple math question
query = "What is 15 * 24?"
answer = run_agent(query)

print("\n" + "="*60)
print("🎯 FINAL ANSWER:")
print("="*60)
print(answer)


📋 AGENT TRACE:
1. [HUMAN] What is 15 * 24?
----------------------------------------
2. [AI] 15 * 24 = 360.
----------------------------------------

🎯 FINAL ANSWER:
15 * 24 = 360.


### Limitations Without Tools

Now let's ask a question that requires **current information** beyond the model's training data:

In [8]:
# Question requiring current information - will show limitation
query = "What are the latest AI breakthroughs announced this week?"
answer = run_agent(query)

print("\n" + "="*60)
print("🎯 FINAL ANSWER:")
print("="*60)
print(answer)
print("\n⚠️ The agent cannot answer this without access to current information tools!")


📋 AGENT TRACE:
1. [HUMAN] What are the latest AI breakthroughs announced this week?
----------------------------------------
2. [AI] I'm not able to provide real-time information or updates on current events, including recent AI breakthroughs. However, I can suggest some reputable sources where you can find the latest news and anno...
----------------------------------------

🎯 FINAL ANSWER:
I'm not able to provide real-time information or updates on current events, including recent AI breakthroughs. However, I can suggest some reputable sources where you can find the latest news and announcements on AI advancements:

1. The New York Times - Technology section
2. MIT Technology Review
3. Wired Magazine
4. arXiv (a preprint server for computer science and related fields)
5. ResearchGate (a social networking platform for scientists and researchers)

You can also check the websites of top AI research institutions, such as:

1. Google AI
2. Microsoft AI
3. Facebook AI
4. DeepMind
5. Stanf

---

## 🔧 Using LangGraph with External Tools

To answer questions beyond the model's training data, we need to connect external tools. Let's add:

1. **DuckDuckGo Search** - For current web information
2. **Wikipedia** - For factual, encyclopedic knowledge

### Enhanced Agent Graph with Tools

```
Basic Agent (No Tools)          Agent with Tools
                                
__start__                     __start__
    │                             │
    ▼                             ▼
  agent                         agent ──────┐
    │                          ↙     ↘      │
    ▼                    __end__    tools ──┘
 __end__                              
```                             

The agent can now loop between reasoning and tool use until it has enough information to answer.

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Create DuckDuckGo search tool (free, no API key!)
search_tool = DuckDuckGoSearchRun()

# Create Wikipedia tool with custom settings
wikipedia_wrapper = WikipediaAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=1000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

# Test the tools
print("🔍 Testing DuckDuckGo Search:")
print(search_tool.invoke("LangGraph framework")[:300] + "...")

print("\n" + "="*60)

print("\n📚 Testing Wikipedia:")
print(wikipedia_tool.invoke("Artificial Intelligence")[:300] + "...")

🔍 Testing DuckDuckGo Search:
November 17, 2025 - LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows . It provides a set of tools and libraries that enable users to create, run and optimize large language ... June 13, 2025 - LangGraph i...


📚 Testing Wikipedia:
Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science t...


In [10]:
# Create agent with external tools
tools = [search_tool, wikipedia_tool]
agent_with_tools = create_react_agent(llm, tools)

print("✅ LangGraph Agent with tools created!")
print("\n🧰 Available Tools:")
for tool in tools:
    print(f"  • {tool.name}: {tool.description[:80]}...")

✅ LangGraph Agent with tools created!

🧰 Available Tools:
  • duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions...
  • wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions...


/tmp/ipykernel_1201/1902272832.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_with_tools = create_react_agent(llm, tools)


In [11]:
def run_agent_with_tools(query: str):
    """Function to run the agent with tools and show detailed traces."""
    print(f"\n🔮 Query: {query}")
    print("="*60)
    
    for chunk in agent_with_tools.stream({"messages": [("user", query)]}):
        if "agent" in chunk:
            msg = chunk["agent"]["messages"][0]
            # Check if agent is calling a tool
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tool_call in msg.tool_calls:
                    print(f"\n🔧 Calling Tool: {tool_call['name']}")
                    print(f"   Input: {tool_call['args']}")
            # Print agent's text response
            if msg.content:
                print(f"\n🤔 Agent: {msg.content[:500]}{'...' if len(msg.content) > 500 else ''}")
        
        if "tools" in chunk:
            tool_msg = chunk["tools"]["messages"][0]
            print(f"\n📋 Tool Result ({tool_msg.name}): {tool_msg.content[:200]}...")
    
    print("\n" + "="*60)

### Testing the Agent with Tools

In [17]:
# Now let's ask about current events - the agent should use DuckDuckGo
run_agent_with_tools("What are the latest developments in Protein folding in 2025?")


🔮 Query: What are the latest developments in Protein folding in 2025?

🔧 Calling Tool: wikipedia
   Input: {'query': 'Protein folding 2025'}

📋 Tool Result (wikipedia): Page: AlphaFold
Summary: AlphaFold is an artificial intelligence (AI) program developed by DeepMind, a subsidiary of Alphabet, which performs predictions of protein structure. It is designed using dee...

🤔 Agent: Based on the latest developments in protein folding as of 2025, AlphaFold has continued to improve its accuracy and performance. Here are some key updates:

1. **AlphaFold 3**: Released in 2022, AlphaFold 3 is an updated version of the original program, with improved performance and accuracy. It achieved a new record score of 95.4% on the CASP15 global distance test (GDT), surpassing its predecessor.
2. **Improved accuracy for difficult targets**: AlphaFold has shown significant improvement in p...



In [18]:
# Factual question - the agent might use Wikipedia
run_agent_with_tools("Who invented the World Wide Web and when?")


🔮 Query: Who invented the World Wide Web and when?

🔧 Calling Tool: wikipedia
   Input: {'query': 'inventor of world wide web'}

📋 Tool Result (wikipedia): Page: World Wide Web
Summary: The World Wide Web (also known as WWW, W3, or simply the Web) is an information system that enables content sharing over the Internet using a graphical user interface. It...

🤔 Agent: Based on the tool call response, I can format an answer to the original user question:

The World Wide Web was invented by Tim Berners-Lee, a British computer scientist, in 1989 while he was working at CERN. The web was initially opened to the public in 1993 and was conceived as a "universal linked information system".



In [19]:
# Complex question requiring search
run_agent_with_tools("What are the top 3 programming languages in 2025? find on internet")


🔮 Query: What are the top 3 programming languages in 2025? find on internet

🔧 Calling Tool: duckduckgo_search
   Input: {'query': 'top programming languages 2025 internet'}

📋 Tool Result (duckduckgo_search): Join the conversation to explore 2025 ’s ranking of Top Programming Languages and get a glimpse into how AI will impact the ranking in 2026–or end it entirely. But programmers are turning away from ma...

🤔 Agent: Based on the search results, the top 3 programming languages in 2025 are:

1. **Python**: Known for its simplicity, readability, and versatility, Python is a popular choice among developers.
2. **JavaScript**: With the rise of web development and mobile app creation, JavaScript has become an essential language for any aspiring developer.
3. **Java**: A versatile language used in Android app development, web development, and enterprise software development.

These languages are expected to be in ...



---

## 🛠️ Creating Custom Tools

One of the most powerful features is creating **custom tools**. This allows your agent to:
- Perform calculations
- Access databases
- Call your own APIs
- Execute any Python code!

### The @tool Decorator

The `@tool` decorator converts any Python function into a LangChain tool. The function's:
- **Name** becomes the tool name
- **Docstring** becomes the tool description (VERY IMPORTANT!)
- **Type hints** help the agent understand input/output types

In [20]:
from langchain.tools import tool

@tool
def find_recipe(dish_name: str) -> str:
    """
    Find a recipe for a given dish. Returns ingredients and cooking instructions.
    
    Args:
        dish_name: The name of the dish to find a recipe for (e.g., 'pasta carbonara', 'chicken curry')
    
    Returns:
        A string containing the recipe with ingredients and instructions.
    """
    # Simulated recipe database
    recipes = {
        "pasta carbonara": {
            "ingredients": ["400g spaghetti", "200g pancetta", "4 egg yolks", "100g Pecorino cheese", "Black pepper"],
            "instructions": "1. Cook pasta. 2. Fry pancetta until crispy. 3. Mix egg yolks with cheese. 4. Combine hot pasta with pancetta, then quickly mix in egg mixture. 5. Season with pepper."
        },
        "chicken curry": {
            "ingredients": ["500g chicken", "2 onions", "3 tbsp curry powder", "400ml coconut milk", "2 tomatoes"],
            "instructions": "1. Sauté onions. 2. Add chicken and brown. 3. Add curry powder and cook 1 min. 4. Add coconut milk and tomatoes. 5. Simmer 20 minutes."
        },
        "chocolate brownies": {
            "ingredients": ["200g dark chocolate", "200g butter", "300g sugar", "3 eggs", "100g flour"],
            "instructions": "1. Melt chocolate and butter. 2. Beat eggs with sugar. 3. Combine mixtures and fold in flour. 4. Bake at 180°C for 25 minutes."
        }
    }
    
    dish_lower = dish_name.lower()
    
    # Try to find a matching recipe
    for key, recipe in recipes.items():
        if key in dish_lower or dish_lower in key:
            ingredients = "\n  - ".join(recipe["ingredients"])
            return (
                f"🍽️ Recipe: {dish_name.title()}\n\n"
                f"📝 Ingredients:\n  - {ingredients}\n\n"
                f"👨‍🍳 Instructions:\n{recipe['instructions']}"
            )
    
    return f"Sorry, I don't have a recipe for '{dish_name}'. Try: pasta carbonara, chicken curry, or chocolate brownies."

# Test the recipe tool
print(find_recipe.invoke({"dish_name": "pasta carbonara"}))

🍽️ Recipe: Pasta Carbonara

📝 Ingredients:
  - 400g spaghetti
  - 200g pancetta
  - 4 egg yolks
  - 100g Pecorino cheese
  - Black pepper

👨‍🍳 Instructions:
1. Cook pasta. 2. Fry pancetta until crispy. 3. Mix egg yolks with cheese. 4. Combine hot pasta with pancetta, then quickly mix in egg mixture. 5. Season with pepper.


In [21]:
@tool
def dictionary_lookup(word: str) -> str:
    """
    Look up a word in the dictionary to get its definition, synonyms, and example usage.
    
    Args:
        word: The word to look up in the dictionary
    
    Returns:
        A string containing the definition, synonyms, and example usage.
    """
    # Simulated dictionary
    dictionary = {
        "ephemeral": {
            "definition": "Lasting for a very short time",
            "synonyms": ["temporary", "fleeting", "transient", "momentary"],
            "example": "The ephemeral beauty of cherry blossoms lasts only a few weeks."
        },
        "serendipity": {
            "definition": "The occurrence of events by chance in a happy or beneficial way",
            "synonyms": ["luck", "fortune", "chance", "coincidence"],
            "example": "Finding that rare book was pure serendipity."
        },
        "ubiquitous": {
            "definition": "Present, appearing, or found everywhere",
            "synonyms": ["omnipresent", "everywhere", "universal", "pervasive"],
            "example": "Smartphones have become ubiquitous in modern society."
        },
        "pragmatic": {
            "definition": "Dealing with things sensibly and realistically",
            "synonyms": ["practical", "realistic", "sensible", "down-to-earth"],
            "example": "We need a pragmatic approach to solve this problem."
        }
    }
    
    word_lower = word.lower().strip()
    
    if word_lower in dictionary:
        entry = dictionary[word_lower]
        synonyms = ", ".join(entry["synonyms"])
        return (
            f"📖 Word: {word.title()}\n\n"
            f"📝 Definition: {entry['definition']}\n\n"
            f"🔄 Synonyms: {synonyms}\n\n"
            f"💬 Example: {entry['example']}"
        )
    
    return f"Sorry, '{word}' is not in my dictionary. Try: ephemeral, serendipity, ubiquitous, or pragmatic."

# Test the dictionary tool
print(dictionary_lookup.invoke({"word": "serendipity"}))

📖 Word: Serendipity

📝 Definition: The occurrence of events by chance in a happy or beneficial way

🔄 Synonyms: luck, fortune, chance, coincidence

💬 Example: Finding that rare book was pure serendipity.


In [22]:
@tool
def calculate_tip(bill_amount: float, tip_percentage: float, num_people: int = 1) -> str:
    """
    Calculate the tip and split the bill among people.
    
    Args:
        bill_amount: The total bill amount in dollars
        tip_percentage: The tip percentage (e.g., 15 for 15%)
        num_people: Number of people to split the bill (default: 1)
    
    Returns:
        A string with tip amount, total bill, and per-person cost.
    """
    tip_amount = bill_amount * (tip_percentage / 100)
    total_bill = bill_amount + tip_amount
    per_person = total_bill / num_people
    
    return (
        f"💰 Bill Calculator:\n\n"
        f"Original Bill: ${bill_amount:.2f}\n"
        f"Tip ({tip_percentage}%): ${tip_amount:.2f}\n"
        f"Total: ${total_bill:.2f}\n"
        f"Per Person ({num_people}): ${per_person:.2f}"
    )

# Test the tip calculator
print(calculate_tip.invoke({"bill_amount": 85.50, "tip_percentage": 20, "num_people": 4}))

💰 Bill Calculator:

Original Bill: $85.50
Tip (20.0%): $17.10
Total: $102.60
Per Person (4): $25.65


### Creating Multi-Tool Agent

Now let's combine all our tools into a powerful multi-purpose agent!

In [23]:
# Combine ALL tools: search, wikipedia, and custom tools
all_tools = [
    search_tool,
    wikipedia_tool,
    find_recipe,
    dictionary_lookup,
    calculate_tip
]

# Print available tools
print("🧰 Available Tools for Multi-Purpose Agent:\n")
for tool in all_tools:
    desc = tool.description.split('\n')[0]  # First line only
    print(f"  • {tool.name}: {desc[:60]}...")
    print("-"*60)

🧰 Available Tools for Multi-Purpose Agent:

  • duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need...
------------------------------------------------------------
  • wikipedia: A wrapper around Wikipedia. Useful for when you need to answ...
------------------------------------------------------------
  • find_recipe: Find a recipe for a given dish. Returns ingredients and cook...
------------------------------------------------------------
  • dictionary_lookup: Look up a word in the dictionary to get its definition, syno...
------------------------------------------------------------
  • calculate_tip: Calculate the tip and split the bill among people....
------------------------------------------------------------


In [24]:
# Create the multi-tool agent
multi_tool_agent = create_react_agent(llm, all_tools)

print("✅ Multi-Tool Agent ready!")

✅ Multi-Tool Agent ready!


/tmp/ipykernel_1201/2806867473.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  multi_tool_agent = create_react_agent(llm, all_tools)


In [25]:
def run_multi_agent(query: str):
    """Run the multi-tool agent with detailed output."""
    print(f"\n🔮 Query: {query}")
    print("="*60)
    
    for chunk in multi_tool_agent.stream({"messages": [("user", query)]}):
        if "agent" in chunk:
            msg = chunk["agent"]["messages"][0]
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tc in msg.tool_calls:
                    print(f"\n🔧 Calling Tool: {tc['name']}")
                    print(f"   Args: {tc['args']}")
            if msg.content:
                print(f"\n🤔 Agent: {msg.content}")
        
        if "tools" in chunk:
            tool_msg = chunk["tools"]["messages"][0]
            print(f"\n📋 Tool Result ({tool_msg.name}):\n{tool_msg.content[:300]}{'...' if len(tool_msg.content) > 300 else ''}")
    
    print("\n" + "="*60)

In [27]:
# Test with recipe request
run_multi_agent("I want to make pasta carbonara tonight. Can you give me the recipe?")


🔮 Query: I want to make pasta carbonara tonight. Can you give me the recipe?

🔧 Calling Tool: find_recipe
   Args: {'dish_name': 'pasta carbonara'}

📋 Tool Result (find_recipe):
🍽️ Recipe: Pasta Carbonara

📝 Ingredients:
  - 400g spaghetti
  - 200g pancetta
  - 4 egg yolks
  - 100g Pecorino cheese
  - Black pepper

👨‍🍳 Instructions:
1. Cook pasta. 2. Fry pancetta until crispy. 3. Mix egg yolks with cheese. 4. Combine hot pasta with pancetta, then quickly mix in egg mixture....

🤔 Agent: Here's the formatted answer:

**Pasta Carbonara Recipe**

To make a delicious Pasta Carbonara, you'll need the following ingredients:

* 400g spaghetti
* 200g pancetta
* 4 egg yolks
* 100g Pecorino cheese
* Black pepper

Now, here's how to prepare your Pasta Carbonara:

1. **Cook pasta**: Bring a large pot of salted water to a boil and cook the spaghetti according to package instructions until al dente.
2. **Fry pancetta**: In a large skillet, cook the pancetta over medium heat until crispy. Remove fro

In [28]:
# Test with dictionary lookup
run_multi_agent("What does the word 'ephemeral' mean? Give me synonyms too.")


🔮 Query: What does the word 'ephemeral' mean? Give me synonyms too.

🔧 Calling Tool: dictionary_lookup
   Args: {'word': 'ephemeral'}

📋 Tool Result (dictionary_lookup):
📖 Word: Ephemeral

📝 Definition: Lasting for a very short time

🔄 Synonyms: temporary, fleeting, transient, momentary

💬 Example: The ephemeral beauty of cherry blossoms lasts only a few weeks.

🤔 Agent: The word "ephemeral" refers to something that is transitory or lasting for a very short period of time. It can also describe experiences, emotions, or phenomena that are fleeting and impermanent.

Some synonyms for "ephemeral" include:

* Temporary
* Fleeting
* Transient
* Momentary

These words all convey the idea that something is not lasting or enduring, but rather is brief and short-lived.



In [29]:
# Test with tip calculation
run_multi_agent("Our dinner bill is $127.50. We want to leave a 18% tip and split it 3 ways. How much does each person pay?")


🔮 Query: Our dinner bill is $127.50. We want to leave a 18% tip and split it 3 ways. How much does each person pay?

🔧 Calling Tool: calculate_tip
   Args: {'bill_amount': '127.5', 'num_people': '3', 'tip_percentage': '18'}

📋 Tool Result (calculate_tip):
💰 Bill Calculator:

Original Bill: $127.50
Tip (18.0%): $22.95
Total: $150.45
Per Person (3): $50.15

🤔 Agent: Each person will pay $50.15.



---

## 💾 Using LangGraph with Memory

So far, our agents have treated every query as an **isolated request**. They cannot:
- Carry on a conversation
- Track context across multiple turns
- Refer to earlier information

To support **back-and-forth interactions**, we need to add **memory** to our agent.

### Understanding State in LangGraph

LangGraph uses a `State` class to maintain conversation history:

```python
class State(TypedDict):
    messages: Annotated[List, add_messages]
```

- `messages` stores the conversation history
- `add_messages` tells LangGraph how to merge, append, or trim messages
- The state persists across multiple agent invocations

In [30]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

# Define State class for memory
class State(TypedDict):
    messages: Annotated[List, add_messages]

print("✅ State class defined for memory management!")

✅ State class defined for memory management!


In [31]:
def run_agent_with_memory(query: str, state: dict = None) -> tuple:
    """
    Run the agent with memory, maintaining conversation history.
    
    Args:
        query: The user's question
        state: Previous conversation state (or None for new conversation)
    
    Returns:
        Tuple of (answer, updated_state)
    """
    # Initialize empty state if none provided
    if state is None:
        state = {"messages": []}
    
    # Append the new user query to existing messages
    state["messages"].append(("user", query))
    
    # Invoke the agent with the full conversation history
    response = multi_tool_agent.invoke(state)
    
    # Update state with the response messages
    state = {"messages": response["messages"]}
    
    # Return the last message (agent's answer) and updated state
    return response["messages"][-1].content, state

print("✅ Memory-enabled agent function ready!")

✅ Memory-enabled agent function ready!


### Testing Conversational Memory

In [32]:
# Start a new conversation
conversation_state = None

# First question
query1 = "What is Python programming language?"
answer1, conversation_state = run_agent_with_memory(query1, conversation_state)

print(f"🧑 User: {query1}")
print(f"\n🤖 Agent: {answer1}")
print("\n" + "="*60)

🧑 User: What is Python programming language?

🤖 Agent: Python is a high-level, general-purpose programming language that supports multiple programming paradigms, including structured (particularly procedural), object-oriented, and functional programming. It was created by Guido van Rossum in the late 1980s as a successor to the ABC programming language. Python's design philosophy emphasizes code readability with the use of significant indentation, and it is dynamically type-checked and garbage-collected.

Some key features of Python include:

* High-level syntax: Python has a simple and easy-to-read syntax that makes it accessible to developers of all skill levels.
* Dynamic typing: Python is dynamically typed, which means that you don't need to declare the data type of a variable before using it.
* Object-oriented programming: Python supports object-oriented programming (OOP) concepts such as classes, objects, inheritance, and polymorphism.
* Large standard library: Python has a vast 

In [33]:
# Follow-up question using context from previous answer
query2 = "Who created it and when?"
answer2, conversation_state = run_agent_with_memory(query2, conversation_state)

print(f"🧑 User: {query2}")
print(f"\n🤖 Agent: {answer2}")
print("\n💡 Notice: The agent understood 'it' refers to Python from the previous question!")
print("\n" + "="*60)

🧑 User: Who created it and when?

🤖 Agent: Guido van Rossum created Python in December 1989 at CWI (Centrum Wiskunde & Informatica) in the Netherlands. He was the principal author of Python and had a central role in deciding its direction until he stepped down as leader on July 12, 2018.

Python was named after the BBC TV show Monty Python's Flying Circus, which Guido van Rossum enjoyed watching. The name was chosen because it was short, unique, and not already associated with any other programming language.

Guido van Rossum's goal in creating Python was to create a scripting language that could be easily learned and used by people of all skill levels. He wanted to create a language that would be versatile enough to be used for a wide range of applications, from simple scripts to complex programs.

Over the years, Guido van Rossum has been involved in many aspects of Python's development, including its design, implementation, and maintenance. He has also been instrumental in shaping t

In [34]:
# Another follow-up
query3 = "What are its main uses today?"
answer3, conversation_state = run_agent_with_memory(query3, conversation_state)

print(f"🧑 User: {query3}")
print(f"\n🤖 Agent: {answer3}")
print("\n" + "="*60)

🧑 User: What are its main uses today?

🤖 Agent: Python is a versatile programming language with a wide range of applications across various industries. Some of its main uses today include:

1. **Web Development**: Python is widely used for web development, especially with frameworks like Django and Flask, which provide a robust and scalable way to build web applications.
2. **Data Science and Machine Learning**: Python is a popular choice for data science and machine learning tasks due to its extensive libraries, such as NumPy, pandas, and scikit-learn, which make it easy to perform data analysis, visualization, and modeling.
3. **Automation**: Python is widely used for automating tasks, such as data processing, file management, and system administration, due to its simplicity and flexibility.
4. **Scientific Computing**: Python is used in various scientific computing applications, including numerical analysis, signal processing, and visualization, thanks to libraries like NumPy, SciPy

### Interactive Chat with Memory

In [36]:
def interactive_chat():
    """
    Start an interactive chat session with the agent.
    Type 'quit' or 'exit' to end the conversation.
    Type 'clear' to start a new conversation.
    """
    print("\n" + "="*60)
    print("🤖 LangGraph Agent with Memory - Interactive Chat")
    print("="*60)
    print("\nCommands:")
    print("  • Type your question to chat")
    print("  • 'clear' - Start new conversation")
    print("  • 'quit' or 'exit' - End chat")
    print("\n" + "-"*60)
    
    conversation_state = None
    
    while True:
        try:
            query = input("\n🧑 You: ").strip()
            
            if not query:
                continue
            
            if query.lower() in ['quit', 'exit']:
                print("\n👋 Goodbye! Thanks for chatting!")
                break
            
            if query.lower() == 'clear':
                conversation_state = None
                print("\n🔄 Conversation cleared. Starting fresh!")
                continue
            
            answer, conversation_state = run_agent_with_memory(query, conversation_state)
            print(f"\n🤖 Agent: {answer}")
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat interrupted. Goodbye!")
            break

# Uncomment the line below to cstart interactive chat:
interactive_chat()


🤖 LangGraph Agent with Memory - Interactive Chat

Commands:
  • Type your question to chat
  • 'clear' - Start new conversation
  • 'quit' or 'exit' - End chat

------------------------------------------------------------



🧑 You:  what is python



🤖 Agent: **What is Python?**

Python is a high-level, general-purpose programming language that was created by Guido van Rossum in the late 1980s as a successor to the ABC programming language. It's known for its simplicity, readability, and ease of use, making it a popular choice for beginners and experienced programmers alike.

**Key Features:**

*   **High-level language**: Python is a high-level language, meaning it abstracts away many low-level details, allowing you to focus on the logic of your program without worrying about memory management or other details.
*   **Dynamic typing**: Python is dynamically typed, which means you don't need to declare the type of a variable before using it. This makes it easier to write code and reduces the amount of boilerplate code you need to write.
*   **Object-oriented programming**: Python supports object-oriented programming (OOP) concepts like classes, objects, inheritance, polymorphism, and encapsulation.
*   **Large standard library**: P


🧑 You:  who created it



🤖 Agent: **Who Created Python?**

Python was created by Guido van Rossum, a Dutch programmer. He began working on Python in the late 1980s and released the first version of the language in December 1991.

**Guido van Rossum's Background:**

*   Born on January 31, 1956, in the Netherlands
*   Studied mathematics and computer science at the University of Amsterdam
*   Worked as a researcher at the Centrum Wiskunde & Informatica (CWI) in Amsterdam
*   Created Python as a successor to the ABC programming language

**Guido van Rossum's Role in Python:**

*   Principal author of Python
*   Led the development of Python until 2018
*   Known for his role as the "Benevolent Dictator for Life" (BDFL) of the Python community
*   Stepped down as leader on July 12, 2018

**Legacy:**

Guido van Rossum's creation of Python has had a significant impact on the programming world. Python is now one of the most popular programming languages in the world, widely used in various fields such as web develop


🧑 You:  exit



👋 Goodbye! Thanks for chatting!


---

## 📝 Summary

In this notebook, we learned how to build **AI Agents** using **LangGraph** with local LLMs (Ollama):

### Key Takeaways

1. **LangGraph** structures agent logic as a **directed graph**:
   - Nodes = computational steps (LLM calls, tool use)
   - Edges = workflow transitions based on state/output

2. **ReAct Pattern** enables step-by-step reasoning:
   - Thought → Action → Observation → Repeat until solved

3. **Tools** extend agent capabilities:
   - Built-in: DuckDuckGo, Wikipedia (free, no API keys!)
   - Custom: Use `@tool` decorator on Python functions

4. **Memory** enables conversational context:
   - `State` class maintains message history
   - `add_messages` handles merging and appending

### Best Practices

| Practice | Why It Matters |
|----------|----------------|
| Write clear tool descriptions | Agents use descriptions to decide which tool to use |
| Use type hints | Helps agents understand input/output formats |
| Include docstrings | Provides context for tool selection |
| Maintain state properly | Enables context-aware conversations |
| Use local LLMs (Ollama) | Privacy, cost savings, no rate limits |

### Graph Comparison

```
Basic Agent (No Tools)          Agent with Tools
                                
__start__                     __start__
    │                             │
    ▼                             ▼
  agent                         agent ──────┐
    │                          ↙     ↘      │
    ▼                    __end__    tools ──┘
 __end__                              
```

### Next Steps

- Explore **LangGraph Studio** for visualizing and debugging agents
- Build **multi-agent systems** where agents collaborate
- Add **persistent storage** for long-term memory across sessions
- Create agents that can **write and execute code**
- Integrate with **databases** and **external APIs**

---

## 🧹 Cleanup (Optional)

If you want to free up resources:

In [38]:
# # Clear variables to free memory
# del multi_tool_agent
# del agent_with_tools
# del agent_executor